In [1]:
import pandas as pd
import sqlite3
import os
import datetime
import time

In [2]:
cur_dir = os.getcwd()
parent_dir = os.path.split(cur_dir)[0]
src_data_path = os.path.join(parent_dir, 'Library', 'Messages', 'chat.db')
src_data_path

'/Users/patrickdwyer/Library/Messages/chat.db'

In [3]:
# substitute username with your username
conn = sqlite3.connect(src_data_path)
# connect to the database
cur = conn.cursor()
# get the names of the tables in the database
cur.execute(" select name from sqlite_master where type = 'table' ")
names = []
for name in cur.fetchall():
    print(name)
    names.append(name)

('_SqliteDatabaseProperties',)
('deleted_messages',)
('sqlite_sequence',)
('chat_handle_join',)
('sync_deleted_messages',)
('message_processing_task',)
('handle',)
('sync_deleted_chats',)
('message_attachment_join',)
('sync_deleted_attachments',)
('kvtable',)
('chat_message_join',)
('message',)
('chat',)
('attachment',)
('sqlite_stat1',)


In [4]:
# get the entries of the message table using pandas
messages = pd.read_sql_query("select * from message", conn)
messages.head()
messages.columns

Index(['ROWID', 'guid', 'text', 'replace', 'service_center', 'handle_id',
       'subject', 'country', 'attributedBody', 'version', 'type', 'service',
       'account', 'account_guid', 'error', 'date', 'date_read',
       'date_delivered', 'is_delivered', 'is_finished', 'is_emote',
       'is_from_me', 'is_empty', 'is_delayed', 'is_auto_reply', 'is_prepared',
       'is_read', 'is_system_message', 'is_sent', 'has_dd_results',
       'is_service_message', 'is_forward', 'was_downgraded', 'is_archive',
       'cache_has_attachments', 'cache_roomnames', 'was_data_detected',
       'was_deduplicated', 'is_audio_message', 'is_played', 'date_played',
       'item_type', 'other_handle', 'group_title', 'group_action_type',
       'share_status', 'share_direction', 'is_expirable', 'expire_state',
       'message_action_type', 'message_source', 'associated_message_guid',
       'associated_message_type', 'balloon_bundle_id', 'payload_data',
       'expressive_send_style_id', 'associated_message_r

In [5]:
# get the handles to apple-id mapping table
handles = pd.read_sql_query("select * from handle", conn)
handles.head()

,ROWID,id,country,service,uncanonicalized_id,person_centric_id
0,1,+19175651968,us,iMessage,9175651968,None
1,2,+13124054400,us,iMessage,3124054400,None
2,3,+19255886132,US,iMessage,None,None
3,4,+12487610953,us,iMessage,2487610953,None
4,5,+16504250933,US,iMessage,None,None


In [6]:
# and join to the messages, on handle_id
messages.rename(columns={'ROWID' : 'message_id'}, inplace = True)
handles.rename(columns={'id' : 'phone_number', 'ROWID': 'handle_id'}, inplace = True)
temp = pd.merge(messages[['text', 'handle_id', 'date','is_sent', 'message_id', 'type', 'account_guid', 'guid', 'associated_message_type', 'associated_message_guid', 'message_action_type', 'message_source']],  handles[['handle_id', 'phone_number']], on ='handle_id', how='left')
temp.message_source.describe()

count    387952.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: message_source, dtype: float64

In [7]:
messages.columns

Index(['message_id', 'guid', 'text', 'replace', 'service_center', 'handle_id',
       'subject', 'country', 'attributedBody', 'version', 'type', 'service',
       'account', 'account_guid', 'error', 'date', 'date_read',
       'date_delivered', 'is_delivered', 'is_finished', 'is_emote',
       'is_from_me', 'is_empty', 'is_delayed', 'is_auto_reply', 'is_prepared',
       'is_read', 'is_system_message', 'is_sent', 'has_dd_results',
       'is_service_message', 'is_forward', 'was_downgraded', 'is_archive',
       'cache_has_attachments', 'cache_roomnames', 'was_data_detected',
       'was_deduplicated', 'is_audio_message', 'is_played', 'date_played',
       'item_type', 'other_handle', 'group_title', 'group_action_type',
       'share_status', 'share_direction', 'is_expirable', 'expire_state',
       'message_action_type', 'message_source', 'associated_message_guid',
       'associated_message_type', 'balloon_bundle_id', 'payload_data',
       'expressive_send_style_id', 'associated_mess

In [8]:
# get the chat to message mapping
chat_message_joins = pd.read_sql_query("select * from chat_message_join", conn)
# and join back to the temp table
chat_message_joins

,chat_id,message_id,message_date
0,984,1,654521310367000064
1,984,2,654507291638843008
2,984,3,654507269185949312
3,984,4,654503092986610432
4,984,5,654502886531523968
...,...,...,...
381539,976,388495,682899212743016960
381540,976,388496,682899229793999872
381541,976,388497,682899285638714496
381542,976,388499,682899391126107904


In [9]:
message_data = pd.merge(temp, chat_message_joins[['chat_id', 'message_id']], on = 'message_id', how='left')

In [10]:
message_attachment_joins = pd.read_sql_query("select * from message_attachment_join", conn)
message_attachment_joins.columns

Index(['message_id', 'attachment_id'], dtype='object')

In [11]:
attachments = pd.read_sql_query("select * from attachment", conn)
attachments


,ROWID,guid,created_date,start_date,filename,uti,mime_type,transfer_state,is_outgoing,user_info,...,sticker_user_info,attribution_info,hide_attachment,ck_sync_state,ck_server_change_token_blob,ck_record_id,original_guid,sr_ck_sync_state,sr_ck_server_change_token_blob,sr_ck_record_id
0,1,at_0_DB1A8701-6A17-43CD-9EEC-CC9DFEDD5412,623541696,0,None,public.mp3,audio/mpeg,0,0,"b""bplist00\xd3\x01\x02\x03\x04\x05\x06TnameXut...",...,None,None,0,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,be149898d481fbc9ad3eabdbcdf7d7e13b16bfb4c48270...,at_0_DB1A8701-6A17-43CD-9EEC-CC9DFEDD5412,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,be149898d481fbc9ad3eabdbcdf7d7e13b16bfb4c48270...
1,2,at_3_4C85FF55-8E3C-43F2-AA70-519F07B59468,623529614,0,None,dyn.age81a5dzq7y066dbtf0g82peqf4hk2pdrb00n5xy,None,0,1,None,...,None,None,1,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,c7b4fc8a02e91929d6dcc7d37d5b63f7fc38bdc5be87f2...,at_3_4C85FF55-8E3C-43F2-AA70-519F07B59468,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,c7b4fc8a02e91929d6dcc7d37d5b63f7fc38bdc5be87f2...
2,3,at_2_4C85FF55-8E3C-43F2-AA70-519F07B59468,623529614,0,None,dyn.age81a5dzq7y066dbtf0g82peqf4hk2pdrb00n5xy,None,0,1,None,...,None,None,1,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,81a8a27a91ec34e6742577a39b00bb142b77a395ab31a2...,at_2_4C85FF55-8E3C-43F2-AA70-519F07B59468,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,81a8a27a91ec34e6742577a39b00bb142b77a395ab31a2...
3,4,at_0_4015AFD0-428D-4C62-B7EF-67B94C6DEBD1,622420102,0,None,dyn.age80c65e,None,0,0,"b""bplist00\xd2\x01\x02\x03\x04TnameXuti-type_\...",...,None,None,0,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,4604973d313e5271243029c15609e29b4660c0d6fa598e...,at_0_4015AFD0-428D-4C62-B7EF-67B94C6DEBD1,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,4604973d313e5271243029c15609e29b4660c0d6fa598e...
4,5,at_0_5C3A33B0-BB7E-4856-8510-7DA01813F9F4,622359371,0,None,public.aifc-audio,audio/x-aiff,0,0,"b""bplist00\xd3\x01\x02\x03\x04\x05\x06TnameXut...",...,None,None,0,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,60a87be4e95ff266750bc9634d681d1aa5f20b86dff9fb...,at_0_5C3A33B0-BB7E-4856-8510-7DA01813F9F4,3,b'bplist00\xd4\x01\x02\x03\x04\x05\x06\x07VX$v...,60a87be4e95ff266750bc9634d681d1aa5f20b86dff9fb...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18957,18975,E5DEBE78-07D0-48DC-ADEC-469FA9651747,682887391,0,~/Library/Messages/Attachments/1d/13/E5DEBE78-...,dyn.age81a5dzq7y066dbtf0g82peqf4hk2pdrb00n5xy,None,5,1,None,...,None,None,1,0,None,None,E5DEBE78-07D0-48DC-ADEC-469FA9651747,0,None,None
18958,18976,5DFF3117-D58A-456D-B674-21594F9EF630,682894443,682894443,~/Library/Messages/Attachments/99/09/5DFF3117-...,public.jpeg,image/jpeg,5,0,b'bplist00\xd8\x01\x02\x03\x04\x05\x06\x07\x08...,...,None,b'bplist00\xd3\x01\x02\x03\x04\x0e\rWpgenszcVp...,0,0,None,None,5DFF3117-D58A-456D-B674-21594F9EF630,0,None,None
18959,18977,F040439A-F03E-45DE-9D25-56A54ACF185D,682895968,682895968,~/Library/Messages/Attachments/98/08/F040439A-...,public.jpeg,image/jpeg,5,0,b'bplist00\xd8\x01\x02\x03\x04\x05\x06\x07\x08...,...,None,b'bplist00\xd3\x01\x02\x03\x04\x0e\rWpgenszcVp...,0,0,None,None,F040439A-F03E-45DE-9D25-56A54ACF185D,0,None,None
18960,18978,09CABF61-0B99-496D-8597-6F5856E7C194,682896881,682896882,~/Library/Messages/Attachments/dd/13/09CABF61-...,public.heic,image/heic,5,0,b'bplist00\xdd\x01\x02\x03\x04\x05\x06\x07\x08...,...,None,b'bplist00\xd3\x01\x02\x03\x04\x0e\rWpgenszcVp...,0,0,None,None,09CABF61-0B99-496D-8597-6F5856E7C194,0,None,None


In [16]:
attachments.rename(columns={'ROWID' : 'attachment_id'}, inplace = True)
message_data.columns

Index(['text', 'handle_id', 'date', 'is_sent', 'message_id', 'type',
       'account_guid', 'guid', 'associated_message_type',
       'associated_message_guid', 'message_action_type', 'message_source',
       'phone_number', 'chat_id'],
      dtype='object')

In [17]:
attachment_data = pd.merge(attachments, message_attachment_joins, on = 'attachment_id', how='left')
attachment_data.columns

Index(['attachment_id', 'guid', 'created_date', 'start_date', 'filename',
       'uti', 'mime_type', 'transfer_state', 'is_outgoing', 'user_info',
       'transfer_name', 'total_bytes', 'is_sticker', 'sticker_user_info',
       'attribution_info', 'hide_attachment', 'ck_sync_state',
       'ck_server_change_token_blob', 'ck_record_id', 'original_guid',
       'sr_ck_sync_state', 'sr_ck_server_change_token_blob', 'sr_ck_record_id',
       'message_id'],
      dtype='object')

In [18]:
cream_data = pd.merge(message_data, attachment_data, on = 'message_id', how='left')
cream_data = cream_data[cream_data.chat_id == 976.0]
cream_data.columns
cream_data = cream_data[['text', 'phone_number', 'is_sent', 'message_id', 'chat_id', 'filename', 'uti', 'is_sticker', 'account_guid', 'guid_x', 'guid_y', 'associated_message_type', 'associated_message_guid', 'mime_type', 'original_guid','attachment_id']]
cream_data

,text,phone_number,is_sent,message_id,chat_id,filename,uti,is_sticker,account_guid,guid_x,guid_y,associated_message_type,associated_message_guid,mime_type,original_guid,attachment_id
9,Loved “Foxes live up to 6 years in the wild on...,+19148863062,0,10,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,A82B15E9-4338-445B-9D65-08D1A91A4F56,NaN,2000,p:0/CD8588D6-0143-421C-8C45-40C6F165C6EA,NaN,NaN,NaN
10,Emphasized “Everyone needs to see fox’s video ...,+15164196484,0,11,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,8D7CC1E1-F32F-4908-B6F0-F1FCB5578AD7,NaN,2004,p:0/206435C8-8EFA-46FD-AAE5-154F8C0E16BF,NaN,NaN,NaN
11,Everyone needs to see fox’s video about formal,+19149602412,0,12,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,206435C8-8EFA-46FD-AAE5-154F8C0E16BF,NaN,0,None,NaN,NaN,NaN
12,I have a brother John b,+14132819306,0,13,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,A2B3828B-7347-4C3B-9CAA-365D7B067415,NaN,0,None,NaN,NaN,NaN
13,Get ur head out of the gutter John B,+19145747734,0,14,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,24ABB3BE-4219-4C19-BD1E-1203C4E75CA4,NaN,0,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393077,￼,+19179231765,0,388495,976.0,~/Library/Messages/Attachments/8a/10/A9849584-...,public.jpeg,0.0,A28C8070-099D-4B76-B07A-1D1E20E4557B,974BB7AA-33A6-4D7E-B199-913D51D45AA0,A9849584-64FF-4C1F-8500-97281C9B23A2,0,None,image/jpeg,A9849584-64FF-4C1F-8500-97281C9B23A2,18979.0
393078,Loved an image,NaN,1,388496,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,5FACD33F-B007-45CE-8D65-523808FB7383,NaN,2000,p:0/974BB7AA-33A6-4D7E-B199-913D51D45AA0,NaN,NaN,NaN
393079,Loved an image,+19176801827,0,388497,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,551BC1A4-8447-49E5-887C-8138A99F4514,NaN,2000,p:1/974BB7AA-33A6-4D7E-B199-913D51D45AA0,NaN,NaN,NaN
393080,Laughed at an image,+19739085595,0,388499,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,19C66CEC-F166-479F-964C-E7F536744A24,NaN,2003,p:0/974BB7AA-33A6-4D7E-B199-913D51D45AA0,NaN,NaN,NaN


In [19]:
def phone_number_cleaner(row):
    sent = row.is_sent
    number = row.phone_number
    if str(sent) == '0':
        return number
    else:
        return '+13128410148'


reaction_list = {'Emphasized', 'Loved', 'Liked',
'Laughed at', 'Disliked', 'Questioned'}


def message_cleaner(row):
    msg = row['text']
    temp_msg = str(msg).split('“')[0].strip()
    for reaction in reaction_list:
        if reaction in temp_msg:
            return reaction
    return 'none'


cream_data.phone_number = cream_data.apply(lambda x: phone_number_cleaner(x), axis = 1)
cream_data['reaction_type'] = cream_data.apply(lambda x: message_cleaner(x), axis = 1)
cream_data

,text,phone_number,is_sent,message_id,chat_id,filename,uti,is_sticker,account_guid,guid_x,guid_y,associated_message_type,associated_message_guid,mime_type,original_guid,attachment_id,reaction_type
9,Loved “Foxes live up to 6 years in the wild on...,+19148863062,0,10,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,A82B15E9-4338-445B-9D65-08D1A91A4F56,NaN,2000,p:0/CD8588D6-0143-421C-8C45-40C6F165C6EA,NaN,NaN,NaN,Loved
10,Emphasized “Everyone needs to see fox’s video ...,+15164196484,0,11,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,8D7CC1E1-F32F-4908-B6F0-F1FCB5578AD7,NaN,2004,p:0/206435C8-8EFA-46FD-AAE5-154F8C0E16BF,NaN,NaN,NaN,Emphasized
11,Everyone needs to see fox’s video about formal,+19149602412,0,12,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,206435C8-8EFA-46FD-AAE5-154F8C0E16BF,NaN,0,None,NaN,NaN,NaN,none
12,I have a brother John b,+14132819306,0,13,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,A2B3828B-7347-4C3B-9CAA-365D7B067415,NaN,0,None,NaN,NaN,NaN,none
13,Get ur head out of the gutter John B,+19145747734,0,14,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,24ABB3BE-4219-4C19-BD1E-1203C4E75CA4,NaN,0,None,NaN,NaN,NaN,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393077,￼,+19179231765,0,388495,976.0,~/Library/Messages/Attachments/8a/10/A9849584-...,public.jpeg,0.0,A28C8070-099D-4B76-B07A-1D1E20E4557B,974BB7AA-33A6-4D7E-B199-913D51D45AA0,A9849584-64FF-4C1F-8500-97281C9B23A2,0,None,image/jpeg,A9849584-64FF-4C1F-8500-97281C9B23A2,18979.0,none
393078,Loved an image,+13128410148,1,388496,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,5FACD33F-B007-45CE-8D65-523808FB7383,NaN,2000,p:0/974BB7AA-33A6-4D7E-B199-913D51D45AA0,NaN,NaN,NaN,Loved
393079,Loved an image,+19176801827,0,388497,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,551BC1A4-8447-49E5-887C-8138A99F4514,NaN,2000,p:1/974BB7AA-33A6-4D7E-B199-913D51D45AA0,NaN,NaN,NaN,Loved
393080,Laughed at an image,+19739085595,0,388499,976.0,NaN,NaN,NaN,A28C8070-099D-4B76-B07A-1D1E20E4557B,19C66CEC-F166-479F-964C-E7F536744A24,NaN,2003,p:0/974BB7AA-33A6-4D7E-B199-913D51D45AA0,NaN,NaN,NaN,Laughed at


In [59]:
#maps phone numbers to people's names
phone_number_map = {'19148863062': 'Andrew',
'19149602412': 'Goldy', '19175651968': 'Callum', '18472249266': 'Casey',
'18479229091': 'Judd', '19145747734': 'Alec', '16147955289': 'Katz',
'18605082030': 'Josh', '19739085595': 'Fox', '19178551473': 'Beau',
'13128410148': 'Patrick', '13868642668': 'Brock', '12402722900': 'Chuck',
'19176801827': 'Boxer', '12037337385': 'Denardi', '19179231765': 'Kimmel',
'12022571999': 'Alvaro', '14132819306': 'Wilson', 'joshleshem@icloud.com': 'Josh', 
'aggig13@gmail.com': 'Alec', 'zacharyfox22@gmail.com': 'Fox'}

In [60]:
creamer_map = {}

for number in phone_number_map.keys():
    name = phone_number_map[number]
    messages_sent = cream_data[cream_data.phone_number == '+'+number]
    counts = []
    index_list = []
    for reaction in reaction_list:
        reaction_data = messages_sent[messages_sent.reaction_type == reaction]
        counts.append(len(reaction_data))
        index_list.append(reaction)
    index_list.append('No Reaction')
    index_list.append('Total')
    normal_data = messages_sent[messages_sent.reaction_type == 'none']
    counts.append(len(normal_data))
    count = len(messages_sent)
    counts.append(count)
    creamer_map[name] = pd.Series(counts, index_list)

In [61]:
#result_data = pd.DataFrame(data=creamer_map, index = ['Emphasized', 'Loved', 'Liked', 'Laughed at', 'Disliked', 'Questioned', 'No Reaction', 'Total'])
#result_data.to_csv(os.path.join(os.getcwd(), 'Princeton_Data.csv'), encoding='utf-8')
cream_data.phone_number.unique()

array(['+19148863062', '+15164196484', '+19149602412', '+14132819306',
       '+19145747734', '+18479229091', '+12022571999', '+12037337385',
       '+16147955289', '+19179231765', '+18472249266', '+19178551473',
       '+18605082030', '+19739085595', '+13128410148', '+19176801827',
       '+19175651968', '+13868642668', '+12402722900', nan,
       '+19145005868', 'zacharyfox22@gmail.com', 'aggig13@gmail.com',
       'joshleshem@icloud.com'], dtype=object)

In [62]:
love_laugh_data = cream_data[(cream_data.reaction_type == 'Loved') | (cream_data.reaction_type == 'Laughed at')]
#love_laugh_data.to_csv(os.path.join(os.getcwd(), 'test.csv'))

In [64]:
text_love_laugh_counts = {}
for idx, row in love_laugh_data.iterrows():
    msg_id = row['associated_message_guid']
    if not pd.isnull(msg_id):
        msg_id = msg_id.split(':')[-1]
        msg_id = msg_id.split('/')[-1]
        text_love_laugh_counts[msg_id] = text_love_laugh_counts.get(msg_id, 0) + 1
    
count_map = {}
filetext_map = {}
name_map = {}

index = []
for msg_id in text_love_laugh_counts.keys():
    temp_df = cream_data[cream_data.guid_x == msg_id]
    if temp_df.shape[0] == 1:
        count_map[msg_id] = text_love_laugh_counts[msg_id]
        index.append(msg_id)
        number = temp_df.iloc[0].phone_number
        number = number.split('+')[-1].strip()
        if number == 15164196484 or number == 19145005868:
            continue
        name = phone_number_map.get(number, 'unknown')
        name_map[msg_id] = name
        file = temp_df.iloc[0].filename
        text = temp_df.iloc[0].text
        if not pd.isnull(file):
            filetext_map[msg_id] = file
        else:
            filetext_map[msg_id] = text
    

data = {'id_to_count': pd.Series(count_map.values(), index=index), 
       'id_to_content': pd.Series(filetext_map.values(), index=index), 
       'id_to_name': pd.Series(name_map.values(), index=index)}




output_data = pd.DataFrame(data=data, index = text_love_laugh_counts.keys())
output_data.columns


Index(['id_to_count', 'id_to_content', 'id_to_name'], dtype='object')

In [42]:
#output_data.sort_values(by=['id_to_count'], inplace=True)
#output_data.to_csv(os.path.join(os.getcwd(), 'Top_Cream.csv'))

TypeError: sort_values() got an unexpected keyword argument 'reverse'

In [66]:

person_love_laugh_ratios = {}

for name in phone_number_map.values():
    temp_df = output_data[output_data.id_to_name == name]
    temp_df = temp_df.dropna()
    length = len(temp_df)
    love_laugh_sum = temp_df.id_to_count.sum()
    ratio = (love_laugh_sum / length)
    person_love_laugh_ratios[name] = ratio
    print(f"{name}: {ratio}")
    
data_2 = {'Average Loves + Laughs': pd.Series(person_love_laugh_ratios.values(), index=person_love_laugh_ratios.keys())}

output_data_2 = pd.DataFrame(data=data_2, index = person_love_laugh_ratios.keys())

output_data_2.sort_values(by=['Average Loves + Laughs'], inplace=True)

output_data_2.to_csv(os.path.join(os.getcwd(), 'Creamer_Stats.csv'))

#cur.close()
#conn.close()

Andrew: 2.2723880597014925
Goldy: 2.100507614213198
Callum: 2.3924180327868854
Casey: 2.058309037900875
Judd: 2.2779850746268657
Alec: 2.0544127405441275
Katz: 2.937074829931973
Josh: 2.335885167464115
Fox: 2.816901408450704
Beau: 3.0281899109792283
Patrick: 2.777992277992278
Brock: 4.6479591836734695
Chuck: 2.8372591006423984
Boxer: 2.5745257452574526
Denardi: 2.5280898876404496
Kimmel: 2.9350525310410696
Alvaro: 2.142857142857143
Wilson: 2.1348897535667963
Josh: 2.335885167464115
Alec: 2.0544127405441275
Fox: 2.816901408450704


In [ ]:
cur.close()
conn.close()